# Project Proposal: Study of the Links Between Gold Market and US Debt & FED Rates

### This project aims to explore the relationships between the gold market, US debt, and Federal Reserve (FED) rates using machine learning techniques. By analyzing historical data and applying various ML models, we hope to uncover patterns and insights that can inform portfolio management and trading strategies.

Authors : Paul-Antoine FRUCHTENREICH & Bilal BENHANA 
Date : 2024

In [8]:
import requests
import json
import pandas as pd

In [22]:
# Dictionary to map dataset IDs to their corresponding endpoints

dataset_endpoints = {
    'debt_to_penny': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/debt_to_penny',
    'average_interest_rate': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/avg_interest_rates',
    'tips_cpi': 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/tips_cpi_data_detail'
}

def fetch_data(dataset_id, start_date, end_date, sort='record_date'):
    """
    Fetch data from the specified dataset endpoint with optional parameters.

    :param dataset_id: str, ID of the dataset to fetch (e.g., 'debt_to_penny', 'average_interest_rate', 'tips_cpi')
    :param start_date: str, start date for the data in 'YYYY-MM-DD' format
    :param end_date: str, end date for the data in 'YYYY-MM-DD' format
    :param sort: str, field to sort the data by (default is 'record_date')
    :return: dict, JSON response from the API
    """
    if dataset_id not in dataset_endpoints:
        raise ValueError(f"Invalid dataset_id. Available options are: {list(dataset_endpoints.keys())}")

    url = dataset_endpoints[dataset_id]
    params = {
        'filter': f'{sort}:gte:{start_date},{sort}:lte:{end_date}',
        'sort': sort
    }
    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")

    df = pd.DataFrame(json.loads(response.text)['data'])
    return df

# Example usage:
debt_to_penny_df = fetch_data('debt_to_penny', '2020-01-01', '2021-01-01')
average_interest_rate_df = fetch_data('average_interest_rate', '2020-01-01', '2021-01-01')
tips_cpi_df = fetch_data('tips_cpi', '2020-01-01', '2021-01-01', sort='index_date')
debt_to_penny_df.head() # Display the first few rows of the DataFrame

,record_date,debt_held_public_amt,intragov_hold_amt,tot_pub_debt_out_amt,src_line_nbr,record_fiscal_year,record_fiscal_quarter,record_calendar_year,record_calendar_quarter,record_calendar_month,record_calendar_day
0,2020-01-02,17162410209193.34,6009588924637.43,23171999133830.77,1,2020,2,2020,1,01,02
1,2020-01-03,17163322135882.56,6007124192808.07,23170446328690.63,1,2020,2,2020,1,01,03
2,2020-01-06,17164653477030.90,6012491408110.99,23177144885141.89,1,2020,2,2020,1,01,06
3,2020-01-07,17156739031334.51,6020202093992.42,23176941125326.93,1,2020,2,2020,1,01,07
4,2020-01-08,17157849214410.19,6004903652551.12,23162752866961.31,1,2020,2,2020,1,01,08
